Lab ML for Data Science: Part II

In [1]:
import numpy as np
import pandas as pd
import torch,torch.optim
import scipy,scipy.spatial
import sklearn,sklearn.datasets
from sklearn.model_selection import train_test_split
import seaborn as sns
from matplotlib import pyplot as plt

# 1 The QM7 Dataset

The QM7 dataset consists of 7165 organic molecules, each of which is composed of up to 23 atoms. The 3d coordinates of each atom in each molecule are available in the variable R. It is an array of size 7165 ×23 ×3 containing for each molecule and atom a triplet representing the 3d coordinates. The variable Z is an array of size 7165×23 which gives for each molecule and atom of the molecule the corresponding atomic number. An atomic number of 1 corresponds to a hydrogen atom (H), the number 6 corresponds to carbon (C), the numbers 7 and 8 to nitrogen (N) and oxygen (O) respectively, and finally, the number 16 corresponds to sulfur (S). If the number is zero, then it indicates that there is no atom at this index, and the corresponding 3d coordinate should therefore be ignored. This allows for representing in the same array molecules of different sizes. In addition to these geometrical features of the molecule, the dataset also provides for each molecule its atomization energy (computed via quantum-chemical simulation). These atomization energy values are stored in the variable T, an array of size 7165.

TLDR:

3D coordinates of each atom in each molecule: \
R: 7165 x 23 x 3

Atomic number of each atom in each molecule: \
Z: 7165 x 23 
- 1:    Hydrogen(H)
- 6:    Carbon(C)
- 7:    Nitrogen(N)
- 8:    Oxygen(O)
- 16:   Sulfur(S)
- 0:    No atom

Atomization energy values of the molecules: \
T: 7165



In [2]:
dataset = scipy.io.loadmat("qm7.mat")
R = dataset["R"]
Z = dataset["Z"]
T = dataset["T"].flatten()

## 1.1 Visualizing Molecules
There are a variety of libraries for rendering molecular geometries with various degrees of sophistication. A quick and dirty approach is to use the scatter plot function of matplotlib, where each point is an atom (e.g. plotted according to its xy-coordinates and discarding the z-coordinate). Note that bonds are not provided as part of QM7 because they are strictly speaking not needed to infer chemical properties (bonds can be derived from atom coordinates). To better visualize the molecule, one can draw connections between nearby atoms by plotting a line between two atoms if the Euclidean distance between the two is smaller than a fixed threshold.

In [94]:
def atom_distance(ab,xy):
    atom_dist = scipy.spatial.distance.cdist(ab,xy, 'euclidean')
    return atom_dist

xlim = [-10,20]
ylim = [-5,15]

plt.figure(figsize=(20, 20))
#Limiting the plots for the visualization
for visual,atom in enumerate(molecule_xy[:100]):
    atoms_xy = atom[(atom[:,0] != 0.) | (atom[:,1] != 0.), :]
    atoms_x = atoms_xy[:,0]
    atoms_y = atoms_xy[:,1]

    atoms_dist = atom_distance(atoms_xy)
    masked_atoms_dist = np.ma.masked_equal(atoms_dist, 0.)
    threshold_mean = np.mean(atoms_dist)
    threshold = 3.

    for i in range(atoms_dist.shape[0]):
        for j in range(atoms_dist.shape[1]):
            if threshold_mean<atoms_dist[i][j] or threshold<atoms_dist[i][j]:
                atoms_dist[i][j]=0

    ax = plt.subplot(10, 10, visual + 1)
    plt.scatter(atoms_x, atoms_y,marker='.',linewidths=0.01)
    for i in range(atoms_dist.shape[0]):
        for j in range(atoms_dist.shape[1]):
            if atoms_dist[i][j]!=0.:
                plt.plot([atoms_xy[i,0], atoms_xy[j,0]], [atoms_xy[i,1], atoms_xy[j,1]])
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)


plt.tight_layout()
plt.show()

NameError: name 'molecule_xy' is not defined

<Figure size 2000x2000 with 0 Axes>

In [70]:
unique_atom = [1,6,7,8,16]

def encode_int_to_feature_vec(atom_number):
    atom_index = unique_atom.index(atom_number)

    encoded_atom_number = np.zeros(len(unique_atom))
    encoded_atom_number[atom_index] = 1

    return encoded_atom_number

In [5]:
def encode_molecule(molecule):
    encoded_molecule = np.zeros(len(unique_atom))

    for atom in molecule:
        if atom != 0.:
            encoded_molecule += encode_int_to_feature_vec(atom)
        else:
            return encoded_molecule
    
    return encoded_molecule

In [6]:
def create_feature_vector_atoms(molecule_list):
    feature_vector_atoms = np.zeros((molecule_list.shape[0],len(unique_atom)))

    for molecule_index, molecule in enumerate(molecule_list):
        encoded_molecule = encode_molecule(molecule)
        feature_vector_atoms[molecule_index] = encoded_molecule
    return feature_vector_atoms

In [231]:
feature_map_atoms = create_feature_vector_atoms(Z)
target_values = T

#Divide the data into test, validation and training data
X_train, X_temp, T_train, T_temp = train_test_split(feature_map_atoms, target_values,  test_size=0.4, random_state=42)
X_val, X_test, T_val, T_test = train_test_split(X_temp, T_temp,  test_size=0.5, random_state=42)

#Center the training data as well as the target values
#Training Data
X_train_mean = np.mean(X_train, axis=0)
T_train_mean = np.mean(T_train)

XC_train = X_train - X_train_mean
TC_train = T_train - T_train_mean
    
#Validation Data
XC_val = X_val - X_train_mean
#Test Data
XC_test = X_test - X_train_mean

In [232]:
def ridge_regression(xc_train_data, tc_train_data, reg_lambda):
    n = xc_train_data.shape[0]
    # Calculate auto-covariance matrix
    sigma_xx = np.dot(xc_train_data.T, xc_train_data) / n

    # Calculate cross-covariance matrix
    sigma_xt = np.dot(xc_train_data.T, tc_train_data) / n

    # Identity matrix for regularization
    I = np.identity(xc_train_data.shape[1])

    # Ridge regression weights
    w = np.linalg.inv(sigma_xx + reg_lambda * I).dot(sigma_xt)
    return w

In [234]:
lambda_values = [1, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001, 0.0000001]
min_mae_val = 100
best_lambda = 0
weights = []

for lambda_reg in lambda_values:
    w = ridge_regression(XC_train, TC_train, lambda_reg)
    
    #Prediction on validation data
    TC_val_pred = np.dot(XC_val, w)
    T_val_pred = TC_val_pred + T_train_mean
    
    #Evaluate performance on validation set
    mae_val = np.mean(np.abs(T_val - T_val_pred))
    if mae_val < min_mae_val:
        min_mae_val = mae_val
        best_lambda = lambda_reg
        weights = w
    print(f"Mean Absolute Error (MAE) on the validation set : {mae_val:.4f} kcal/molfor lambda = {lambda_reg}")

#Prediction on test data
TC_test_pred = np.dot(XC_test, weights)
T_test_pred = TC_test_pred + T_train_mean

#Evaluate performance on test set
mae_val = np.mean(np.abs(T_test - T_test_pred))
print(f"Mean Absolute Error (MAE) on the test set : {mae_val:.4f} kcal/mol for lambda = {best_lambda}")

Mean Absolute Error (MAE) on the validation set : 46.5199 kcal/molfor lambda = 1
Mean Absolute Error (MAE) on the validation set : 26.3001 kcal/molfor lambda = 0.1
Mean Absolute Error (MAE) on the validation set : 16.5445 kcal/molfor lambda = 0.01
Mean Absolute Error (MAE) on the validation set : 15.4427 kcal/molfor lambda = 0.001
Mean Absolute Error (MAE) on the validation set : 15.4083 kcal/molfor lambda = 0.0001
Mean Absolute Error (MAE) on the validation set : 15.4062 kcal/molfor lambda = 1e-05
Mean Absolute Error (MAE) on the validation set : 15.4060 kcal/molfor lambda = 1e-06
Mean Absolute Error (MAE) on the validation set : 15.4060 kcal/molfor lambda = 1e-07
Mean Absolute Error (MAE) on the test set : 15.7658 kcal/mol for lambda = 1e-07


In [235]:
# Calculate the contribution of each element i for each molecule individually in the test set
for index_mol, endcoded_mol in enumerate(X_test):
    atom_contributions = endcoded_mol * weights
    print(f"Molecule {endcoded_mol} contributions:")
    for atom, contribution in zip(['H', 'C', 'N', 'O', 'S'], atom_contributions):
        print(f"  {atom}: {contribution:.4f}")
    atomization_energy = sum(atom_contributions)
    print(f"The calculated atomization energy of this molecule is: {atomization_energy:.3f} but the actual value should be: {T_test[index_mol]:.3f}")

Molecule [8. 4. 2. 1. 0.] contributions:
  H: -549.3394
  C: -630.3704
  N: -205.3068
  O: -101.3881
  S: -0.0000
The calculated atomization energy of this molecule is: -1486.405 but the actual value should be: -1448.290
Molecule [7. 5. 1. 0. 1.] contributions:
  H: -480.6720
  C: -787.9630
  N: -102.6534
  O: -0.0000
  S: -80.3367
The calculated atomization energy of this molecule is: -1451.625 but the actual value should be: -1444.630
Molecule [12.  6.  0.  1.  0.] contributions:
  H: -824.0091
  C: -945.5556
  N: -0.0000
  O: -101.3881
  S: -0.0000
The calculated atomization energy of this molecule is: -1870.953 but the actual value should be: -1842.090
Molecule [6. 5. 0. 2. 0.] contributions:
  H: -412.0045
  C: -787.9630
  N: -0.0000
  O: -202.7762
  S: -0.0000
The calculated atomization energy of this molecule is: -1402.744 but the actual value should be: -1396.760
Molecule [9. 5. 1. 1. 0.] contributions:
  H: -618.0068
  C: -787.9630
  N: -102.6534
  O: -101.3881
  S: -0.0000
Th

In [74]:
unique_pairs = []
for type1 in unique_atom:
    for type2 in unique_atom:
        if not [type2,type1] in unique_pairs:
            unique_pairs.append([type1,type2])

In [75]:
unique_pairs

[[1, 1],
 [1, 6],
 [1, 7],
 [1, 8],
 [1, 16],
 [6, 6],
 [6, 7],
 [6, 8],
 [6, 16],
 [7, 7],
 [7, 8],
 [7, 16],
 [8, 8],
 [8, 16],
 [16, 16]]

In [88]:
def create_atom_pairs(molecule):
    pairs = []
    for i,atom1 in enumerate(molecule):
        if atom1 != 0.:
            for j,atom2 in enumerate(molecule):
                if not (j,i) in pairs and atom2 != 0. and j != i:
                        pairs.append((i,j))
    
    return np.array(pairs)

In [89]:
create_atom_pairs(Z[0])

array([[0, 1],
       [0, 2],
       [0, 3],
       [0, 4],
       [1, 2],
       [1, 3],
       [1, 4],
       [2, 3],
       [2, 4],
       [3, 4]])

In [76]:
def encode_pair_to_feature_vec(pair):
    pair_index = unique_pairs.index(pair)

    encoded_atom_number = np.zeros(len(unique_pairs))
    encoded_atom_number[pair_index] = 1

    return encoded_atom_number

In [83]:
def encode_molecule_pairs(molecule):
    encoded_molecule_pairs = np.zeros(len(unique_pairs))
    
    atom_pairs = [[molecule[a],molecule[b]] for a,b in create_atom_pairs(molecule)]

    for pairs in atom_pairs:
        encoded_molecule_pairs += encode_pair_to_feature_vec(sorted(pairs))
        
    return encoded_molecule_pairs

In [85]:
def create_feature_vector_molecule_pairs(molecule_list):
    feature_vector_pairs = np.zeros((molecule_list.shape[0],len(unique_pairs)))

    for molecule_index, molecule in enumerate(molecule_list):
        encoded_molecule_pairs = encode_molecule_pairs(molecule)
        feature_vector_pairs[molecule_index] = encoded_molecule_pairs
    return feature_vector_pairs

In [107]:
feature_map_pairs = create_feature_vector_molecule_pairs(Z)

In [99]:
def encode_molecule_distances(index_molecule,molecule):
    atom_pairs = create_atom_pairs(molecule)
    
    encode_molecule_distances = np.zeros(len(atom_pairs))

    for pair_index,(atom1,atom2) in enumerate(atom_pairs):
        encode_molecule_distances[pair_index] = np.linalg.norm(R[index_molecule,atom1]-R[index_molecule,atom2])
    
    return encode_molecule_distances

In [101]:
encode_molecule_distances(0,Z[0])

array([2.06354904, 2.0635345 , 2.06358266, 2.0651567 , 3.37018085,
       3.37019825, 3.37065625, 3.37019253, 3.37065315, 3.37067175])

In [313]:
def gaussian(dist, mean, variance):
    return (1 / np.sqrt(2 * np.pi * variance)) * np.exp(-0.5 * ((dist - mean) ** 2) / variance)

def create_feature_vector_molecule_dist(molecule_list):
    min_dist = 1
    max_dist = 18
    intervalsteps = 0.17
    variance = 0.01
    intervals = np.zeros(round((max_dist -  min_dist)/ intervalsteps), dtype=float)
    l = len(intervals)

    feature_vector_dist = np.zeros((molecule_list.shape[0],l))

    for molecule_index, molecule in enumerate(molecule_list):
        encoded_molecule_dist = encode_molecule_distances(molecule_index,molecule)
        intervals = np.zeros(round((max_dist -  min_dist)/ intervalsteps), dtype=float)
        for dist in encoded_molecule_dist:
            for interval_index in range(l):
                interval_start = min_dist + interval_index * intervalsteps
                interval_end = interval_start + intervalsteps
                interval_center = (interval_start + interval_end) / 2
                weight = gaussian(dist, interval_center, variance)
                if weight == 0.:
                    continue
                intervals[interval_index] += weight
        feature_vector_dist[molecule_index] = intervals

    return feature_vector_dist

In [314]:
feature_map_distances = create_feature_vector_molecule_dist(Z)

In [315]:
def combine_feature_maps(feature_map_pairs, feature_map_dist):
    combined_feature_map = np.zeros((feature_map_pairs.shape[0], feature_map_pairs.shape[1] * feature_map_dist.shape[1]))

    # Compute the combined feature map
    for i in range(feature_map_pairs.shape[0]):
        outer_product = np.outer(feature_map_dist[i], feature_map_pairs[i])
        combined_feature_map[i] = outer_product.flatten()
    return combined_feature_map

In [316]:
combined_map = combine_feature_maps(feature_map_pairs, feature_map_distances)

In [317]:
combined_map

array([[1.48486524e-19, 9.89910162e-20, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [3.89628766e-19, 3.11703013e-19, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [5.03547834e-19, 6.71397111e-19, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [1.36006946e-18, 2.04010418e-18, 3.40017364e-19, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [7.91348430e-18, 1.23098645e-17, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [8.60326756e-18, 1.09496133e-17, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [318]:
big_feature_map = combined_map
target_values = T

#Divide the data into test, validation and training data
X_train, X_temp, T_train, T_temp = train_test_split(big_feature_map, target_values,  test_size=0.4, random_state=42)
X_val, X_test, T_val, T_test = train_test_split(X_temp, T_temp,  test_size=0.5, random_state=42)

#Center the training data as well as the target values
#Training Data
X_train_mean = np.mean(X_train, axis=0)
T_train_mean = np.mean(T_train)

XC_train = X_train - X_train_mean
TC_train = T_train - T_train_mean
    
#Validation Data
XC_val = X_val - X_train_mean
#Test Data
XC_test = X_test - X_train_mean

In [319]:
lambda_values = [100, 10, 1, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001, 0.0000001]
min_mae_val = 100
best_lambda = 0
weights = []

for lambda_reg in lambda_values:
    w = ridge_regression(XC_train, TC_train, lambda_reg)
    
    #Prediction on validation data
    TC_val_pred = np.dot(XC_val, w)
    T_val_pred = TC_val_pred + T_train_mean
    
    #Evaluate performance on validation set
    mae_val = np.mean(np.abs(T_val - T_val_pred))
    if mae_val < min_mae_val:
        min_mae_val = mae_val
        best_lambda = lambda_reg
        weights = w
    print(f"Mean Absolute Error (MAE) on the validation set : {mae_val:.4f} kcal/mol for lambda = {lambda_reg}")

Mean Absolute Error (MAE) on the validation set : 10.3822 kcal/mol for lambda = 100
Mean Absolute Error (MAE) on the validation set : 8.9654 kcal/mol for lambda = 10
Mean Absolute Error (MAE) on the validation set : 9.0884 kcal/mol for lambda = 1
Mean Absolute Error (MAE) on the validation set : 9.4644 kcal/mol for lambda = 0.1
Mean Absolute Error (MAE) on the validation set : 10.0596 kcal/mol for lambda = 0.01
Mean Absolute Error (MAE) on the validation set : 10.6739 kcal/mol for lambda = 0.001
Mean Absolute Error (MAE) on the validation set : 11.0324 kcal/mol for lambda = 0.0001
Mean Absolute Error (MAE) on the validation set : 10.9344 kcal/mol for lambda = 1e-05
Mean Absolute Error (MAE) on the validation set : 11.2279 kcal/mol for lambda = 1e-06
Mean Absolute Error (MAE) on the validation set : 15.3023 kcal/mol for lambda = 1e-07


In [335]:
min_mae_val = 100
best_lambda = 0
weights = []
for lambda_reg in lambda_values:
    w = ridge_regression(XC_train, TC_train, lambda_reg)
    
    #Prediction on test data
    TC_test_pred = np.dot(XC_test, w)
    T_test_pred = TC_test_pred + T_train_mean

    #Evaluate performance on test set
    mae_val = np.mean(np.abs(T_test - T_test_pred))
    if mae_val < min_mae_val:
        min_mae_val = mae_val
        best_lambda = lambda_reg
        weights = w
print(f"Mean Absolute Error (MAE) on the test set : {min_mae_val:.4f} kcal/mol for lambda = {best_lambda}")

Mean Absolute Error (MAE) on the test set : 9.5146 kcal/mol for lambda = 1
